In [3]:
pip install -Uq fastai

Note: you may need to restart the kernel to use updated packages.


In [21]:
from tqdm import tqdm

In [1]:
import fastai
fastai.__version__

'2.6.3'

In [2]:
from fastai.basics import *
from fastai.callback.all import *
from fastai.text.all import *

In [77]:
path = untar_data(URLs.WIKITEXT)

In [78]:
df_train = pd.read_csv(path/'train.csv', header=None)
df_valid = pd.read_csv(path/'test.csv', header=None)
df_all = pd.concat([df_train, df_valid])

In [79]:
splits = [list(range_of(df_train)), list(range(len(df_train), len(df_all)))]
tfms = [attrgetter("text"), Tokenizer.from_df(0), Numericalize()]
dsets = Datasets(df_all, [tfms], splits=splits, dl_type=LMDataLoader)

In [80]:
bs,sl = 5,512
dls = dsets.dataloaders(bs=bs, seq_len=sl)

In [81]:
lm = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=Perplexity(), pretrained=True)

In [82]:
lm.validate()

(#2) [3.2393643856048584,25.517498016357422]

In [83]:
df_memory = pd.read_pickle("balanced_inputs.pkl")
batch_size = 32

In [84]:
def tokenize_fastai(x):
    tokenized = np.zeros((len(df_memory), 512))
    for idx, c in tqdm(enumerate(x)):
        tokens = np.array(dsets.numericalize(dsets.tokenizer(c)).tolist())[:512]
        tokenized[idx, 512-len(tokens):] = tokens
    
    return tokenized  

In [85]:
def rep_fastai(tokenized):
    count = 0
    rep_list = []
    while count < len(df_memory):
        rep_list.append(lm.model[0](
            torch.Tensor(tokenized[count:count+batch_size]).long().cuda()).detach().cpu().numpy()[:,-1,:])
        count += batch_size
        #print(count)
        
    return torch.tensor(np.vstack(rep_list))   

In [86]:
tokenized_context = tokenize_fastai(df_memory['context'])
tokenized_query= tokenize_fastai(df_memory['query'])

387216it [26:43, 241.52it/s]
387216it [02:08, 3006.19it/s]


In [91]:
X_context = rep_fastai(tokenized_context)
X_query = rep_fastai(tokenized_query)

In [92]:
label = torch.tensor(df_memory['label'].values).float()
target = torch.tensor(df_memory['label'].values)

In [93]:
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split

data_set = TensorDataset(X_context,X_query,label, target)

train_len = int(len(data_set)*0.8)
test_len = int(len(data_set)*0.1)
val_len = len(data_set) - train_len - test_len
train_set, val_set, test_set = random_split(data_set, [train_len, val_len, test_len])

train_loader = DataLoader(train_set,batch_size=128,shuffle=True)
test_loader = DataLoader(test_set,batch_size=128,shuffle=False)
val_loader = DataLoader(val_set,batch_size=128,shuffle=False) 

In [94]:
checkpoint_path = '/data/sherin/checkpoint_lm/chkpt_lm_lstm_fastai_recall_best.pt.tar'

In [95]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [96]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.linear = nn.Sequential(
            # 384 is the size of the embedding
            nn.Linear(400, 400)
        )

    def forward(self, x, y):
        #print(x.size())
        x_input = self.linear(x)
        #print(x_input.size())
        #print(y.size())
        op = torch.sum(x_input*y, dim=1)
        #print(op.shape)
        return op

In [97]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear): Sequential(
    (0): Linear(in_features=400, out_features=400, bias=True)
  )
)


In [98]:
import torch.optim as optim

#criterion = nn.functional.binary_cross_entropy()
criterion = nn.BCEWithLogitsLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [99]:
import torchmetrics

epoch_loss_list = []
accuracy_list = []
val_loss_list = []
val_acc_list = []
valid_acc_max = 0 

for epoch in range(20):  # loop over the dataset multiple times
    train_count = 0
    model.train()
    epoch_loss = 0.0
    accuracy = 0.0
    
    #for ind in tqdm(range(len(df_short)))
    for c, q, label, target in tqdm(train_loader):
        train_count = train_count+1
        #context = torch.tensor(model_sent_trans.encode(list(c))).to(device)
        context = c.to(device)
        
        #query = torch.tensor(model_sent_trans.encode(list(q))).to(device)
        query = q.to(device)
        
        target = target.to(device)
        #label = torch.tensor(labels.float()).to(device)
        label = label.to(device)
        #print(context.size())
        #print(query.size())     
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(context, query)
        #loss = criterion(outputs, labels)
        loss = criterion(outputs, label)
        loss.backward()

        optimizer.step()
        #print(loss.item())
        # print statistics
        #running_loss += loss.item()
        epoch_loss += loss.item()
        accuracy += torchmetrics.functional.accuracy(outputs, target, threshold=0.5).item()
        #if i % 2000 == 1999:    # print every 2000 mini-batches
        #print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        #running_loss = 0.0
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    
    test_count = 0
    for c, q, label, target in tqdm(val_loader):
        test_count = test_count + 1
        context = c.to(device)
        query = q.to(device)
        
        target = target.to(device)
        label = label.to(device)
        
        outputs = model(context, query)
        loss = criterion(outputs, label)
        val_loss += loss.item()
        val_acc += torchmetrics.functional.accuracy(outputs, target, threshold=0.5).item()
        
    accuracy = accuracy / train_count
    epoch_loss = epoch_loss / train_count
    val_loss = val_loss / test_count
    val_acc = val_acc / test_count
    
    if val_acc > valid_acc_max:
        print("saving best model")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
            'accuracy': val_acc,
            }, checkpoint_path)
        valid_acc_max = val_acc
    else:
        print("not saving the model")
    
    print(f'[{epoch + 1}] Training loss: {epoch_loss:.3f} Training accuracy : {accuracy:.3f}')
    print(f'[{epoch + 1}] Validation loss: {val_loss:.3f} Validation accuracy : {val_acc:.3f}')
    epoch_loss_list.append(epoch_loss)
    accuracy_list.append(accuracy)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

print('Finished Training')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 303/303 [00:01<00:00, 267.46it/s]


saving best model
[1] Training loss: 0.594 Training accuracy : 0.668
[1] Validation loss: 0.535 Validation accuracy : 0.704


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 303/303 [00:01<00:00, 265.37it/s]


saving best model
[2] Training loss: 0.491 Training accuracy : 0.746
[2] Validation loss: 0.487 Validation accuracy : 0.747


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 303/303 [00:01<00:00, 266.64it/s]


...

not saving the model
[18] Training loss: 0.273 Training accuracy : 0.881
[18] Validation loss: 0.362 Validation accuracy : 0.858


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 303/303 [00:02<00:00, 119.39it/s]


not saving the model
[19] Training loss: 0.270 Training accuracy : 0.884
[19] Validation loss: 0.358 Validation accuracy : 0.848


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 303/303 [00:01<00:00, 272.22it/s]

saving best model
[20] Training loss: 0.266 Training accuracy : 0.885
[20] Validation loss: 0.346 Validation accuracy : 0.861
Finished Training


In [100]:
model = NeuralNetwork().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
PATH = checkpoint_path
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

# inferece
model.eval()

NeuralNetwork(
  (linear): Sequential(
    (0): Linear(in_features=400, out_features=400, bias=True)
  )
)

In [102]:
test_count = 0
output_logits = []
for c, q, label, target in tqdm(test_loader):
    context = c.to(device)
    query = q.to(device)
    target = target.to(device)
    label = label.to(device)
        
    outputs = model(context, query).detach().cpu().numpy()
    output_logits.append(outputs)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 303/303 [00:00<00:00, 405.75it/s]


In [103]:
test_output_logits = np.hstack(output_logits)

In [104]:
df_test_lstm = df_memory.iloc[test_loader.dataset.indices]

In [108]:
test_output_logits.shape

(38721,)

In [110]:
df_test_lstm["pred"] = torch.sigmoid(torch.tensor(test_output_logits))

SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
df_test_lstm["lstm_pred_label"] = 1.0 * (df_test_lstm["bert_pred"] > 0.5)